In [1]:
from tsp_annealing import *
import matplotlib.pyplot as plt

In [9]:
opt_distance_51, opt_distance_280, opt_distance_442 = calculate_optimal_distances()

cities_51 = load_graph('TSP-Configurations/eil51.tsp.txt')
cities_280 = load_graph('TSP-Configurations/a280.tsp.txt')
cities_442 = load_graph('TSP-Configurations/pcb442.tsp.txt')

distances_51 = calculate_distances(cities_51)
distances_280 = calculate_distances(cities_280)
distances_442 = calculate_distances(cities_442)

In [10]:
def plot_TSP_solution(cities, permutation_method, opt_solution=None, **kwargs):
    plt.figure(figsize=(8, 6))
    plt.scatter([elem[0] for elem in cities], [elem[1] for elem in cities])
    plt.title('Map')

    distances = calculate_distances(cities)

    best_tour, best_distance, cost_over_iterations, temperature_over_interations, count = perform_annealing(distances, 
                                                                    altering_method=permutation_method, output_count=True, **kwargs)

    print("Best tour:", best_tour)
    print("Best distance:", best_distance)

    plot_tour(best_tour, cities, permutation_method)
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.plot(cost_over_iterations)
    if opt_solution is not None:
        opt_line = [opt_solution for _ in range(len(cost_over_iterations))]
        plt.plot(opt_line, linestyle='--', color='black')
    plt.title('Tour Distance Over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Distance')
    plt.show()

    plt.plot(temperature_over_interations)
    plt.title('Temperature Over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Temperature')

    plt.show()
    print('annealing iterations:', count)



def metrics(cities, permutation_method, print_results=True, **kwargs):
    distances = calculate_distances(cities)

    best_tour, best_tour_distance, _, _ = perform_annealing(distances, altering_method=permutation_method, **kwargs)

    best_tour_coordinates = tour_to_cities(best_tour, cities)

    if print_results:
        print('Method = \'%s\''%(permutation_method))
        print("Best distance:", best_tour_distance)
        print('With %i intersections \n'%(count_intersections(best_tour_coordinates)))
    
    return best_tour_distance, count_intersections(best_tour_coordinates)

def plot_dist_and_temp(costs, temps, title=None):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].set_title('Distance over iterations')
    axs[0].plot(costs)
    axs[1].set_title('Temperature')
    axs[1].plot(temps)
    if title is not None:
        fig.suptitle(title)
    plt.show()

In [11]:
cities = load_graph('TSP-Configurations/eil51.tsp.txt')

metrics(cities, 'swap')
metrics(cities, 'insert')
metrics(cities, 'reverse')

print('Actual optimal distance:', opt_distance_51)

Method = 'swap'
Best distance: 592.6373900152993
With 10 intersections 

Method = 'insert'
Best distance: 618.2803961715925
With 14 intersections 

Method = 'reverse'
Best distance: 474.8140691132331
With 2 intersections 

Actual optimal distance: 447.79299344253565


In [12]:
cities = load_graph('TSP-Configurations/a280.tsp.txt')

metrics(cities, 'swap')
metrics(cities, 'insert')
metrics(cities, 'reverse')

print('Actual optimal distance:', opt_distance_280)

Method = 'swap'
Best distance: 12786.267810612257
With 1045 intersections 

Method = 'insert'
Best distance: 11161.762345079736
With 824 intersections 

Method = 'reverse'
Best distance: 9326.512092792229
With 498 intersections 

Actual optimal distance: 2586.7696475631606


In [8]:
cities = load_graph('TSP-Configurations/eil51.tsp.txt')
max_iterations = int(1E6)
final_temp = 1E-8
cooling_schedule = 'quadratic_a'
alpha = 1 - 1E-9
distances = calculate_distances(cities)

kwargs_list = [
    {
        'distances': distances,
        'altering_method': 'swap',
        'max_iterations': max_iterations,
        'final_temp': final_temp,
        'cooling_schedule': cooling_schedule,
        'alpha': alpha
    },
    {
        'distances': distances,
        'altering_method': 'insert',
        'max_iterations': max_iterations,
        'final_temp': final_temp,
        'cooling_schedule': cooling_schedule,
        'alpha': alpha
    },
    {
        'distances': distances,
        'altering_method': 'reverse',
        'max_iterations': max_iterations,
        'final_temp': final_temp,
        'cooling_schedule': cooling_schedule,
        'alpha': alpha
    }
]

start_time = time.time()
for kwargs in kwargs_list:
    perform_annealing(**kwargs)
end_time = time.time()
print('Time taken with NO concurrency', end_time - start_time)

output = run_concurrent(perform_annealing, param_sets=kwargs_list)

for i in range(3):    
    print(np.mean(output[i][1]))

Time taken with NO concurrency 66.70474219322205
Time taken with concurrency: 36.03011226654053 seconds
